<a href="https://colab.research.google.com/github/MondaAtef/projects_3th/blob/FCL-C-programming/function_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import numpy as np
import numba
from numba import jit,njit

In [ ]:
df=pd.read_csv("/content/consumers-price-index-march-2023-quarter-index-numbers.csv",na_values='0')

In [ ]:
df

,Series_reference,Period,Data_value,STATUS,UNITS,Subject,Group,Series_title_1
0,CPIQ.SE901,1914.06,13.167858,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
1,CPIQ.SE901,1914.09,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
2,CPIQ.SE901,1914.12,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
3,CPIQ.SE901,1915.03,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
4,CPIQ.SE901,1915.06,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
...,...,...,...,...,...,...,...,...
20570,CPIQ.SE9A,2022.03,1142.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20571,CPIQ.SE9A,2022.06,1161.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20572,CPIQ.SE9A,2022.09,1186.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20573,CPIQ.SE9A,2022.12,1203.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups


### Data cleaning &pre_pareing 

In [ ]:
df.columns

Index(['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS',
       'Subject', 'Group', 'Series_title_1'],
      dtype='object')

In [ ]:
data=df.rename(columns={'Period':'history','Series_title_1':'products','Data_value':'value'})

In [ ]:
data.head()

,Series_reference,history,value,STATUS,UNITS,Subject,Group,products
0,CPIQ.SE901,1914.06,13.167858,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
1,CPIQ.SE901,1914.09,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
2,CPIQ.SE901,1914.12,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
3,CPIQ.SE901,1915.03,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
4,CPIQ.SE901,1915.06,NaN,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food


In [ ]:
 mask= data.isnull().any(axis=1)
 Rows_with_null= mask.sum()
 total=len(data)
 Rows_with_null/total


0.0027217496962332927

In [ ]:
data_clean=data[~mask]
data_clean

,Series_reference,history,value,STATUS,UNITS,Subject,Group,products
0,CPIQ.SE901,1914.06,13.167858,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
5,CPIQ.SE901,1915.09,14.919183,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
9,CPIQ.SE901,1916.09,15.801429,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
13,CPIQ.SE901,1917.09,16.775851,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
17,CPIQ.SE901,1918.09,18.540343,FINAL,Index,CPI,CPI Level 1 Groups for New Zealand,Food
...,...,...,...,...,...,...,...,...
20570,CPIQ.SE9A,2022.03,1142.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20571,CPIQ.SE9A,2022.06,1161.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20572,CPIQ.SE9A,2022.09,1186.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups
20573,CPIQ.SE9A,2022.12,1203.000000,FINAL,Index,CPI,CPI All Groups for New Zealand,All groups


In [ ]:
d = data_clean.groupby(['history','products'])['value'].sum().reset_index()
d.head()
d['rnk'] = d.groupby('products')['value'].rank(method='first',ascending=False)

In [ ]:
d[d['rnk']<=3]

,history,products,value,rnk
1815,1987.03,Purchase of bicycles,1829.682741,3.0
1868,1987.06,Purchase of bicycles,1846.287238,1.0
1871,1987.06,Purchase of new motor cars,1380.708481,1.0
1921,1987.09,Purchase of bicycles,1830.686133,2.0
2637,1990.12,Credit services,1442.110011,3.0
...,...,...,...,...
18934,2023.03,Veterinary services,1271.000000,1.0
18935,2023.03,Vocational services,1098.000000,1.0
18936,2023.03,Water supply,1264.000000,3.0
18937,2023.03,Wine,1152.000000,1.0


In [ ]:
d[(d['rnk'] <= 3)&(d['products'] == 1)]

### function

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features=["products","history"]
one_hot=OneHotEncoder()
transformer=ColumnTransformer([("one_hot",one_hot,categorical_features)],remainder="passthrough")


In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### function without GPU

In [ ]:
def sum_value_for_food(d):

  sum_of_values = d['value'].sum()

  return sum_of_values
         

In [ ]:
%time _ =sum_value_for_food(d)

CPU times: user 83 µs, sys: 937 µs, total: 1.02 ms
Wall time: 968 µs


### the same function but with GPU

In [ ]:
@njit
def sum_value_for_food(arr):
    sum_of_values = arr.sum()
    return sum_of_values


arr = d['value'].to_numpy()  
result = sum_value_for_food(arr)

In [ ]:
%time _ =sum_value_for_food(arr)

CPU times: user 37 µs, sys: 0 ns, total: 37 µs
Wall time: 40.5 µs
